## Simple RNN Implementation

This implementation of a Simple RNN follows the basic structure of recurrent neural networks, designed to process sequential data.

1. **Initialization**:  
   - The weights for input-to-hidden, hidden-to-hidden, and hidden-to-output connections are initialized.
   - Bias terms are added for the hidden and output layers. The weights are scaled appropriately to work well with the $\tanh$ activation function.

2. **Forward Pass**:  
   - For each time step in the sequence:
     - The hidden state is updated using the input, previous hidden state, and hidden weights.
     - The output is calculated based on the current hidden state and the output weights.
   - This process captures temporal dependencies in the sequence.

3. **Backward Pass**:  
   - Gradients of the loss function are computed using backpropagation through time (BPTT).
   - Since each hidden state affects multiple outputs, gradients are accumulated across time steps.
   - The weights and biases are updated using the accumulated gradients to minimize the loss.

4. **Training Loop**:  
   - The network is trained iteratively over multiple epochs, processing the input sequence through forward and backward passes and updating parameters.


In [1]:
import pandas as pd
import numpy as np

# Read in our data, and fill missing values
data = pd.read_csv("clean_weather.csv", index_col=0)
data = data.ffill()

# Display a sequence of temperatures
data["tmax"].head(10)

1970-01-01    60.0
1970-01-02    52.0
1970-01-03    52.0
1970-01-04    53.0
1970-01-05    52.0
1970-01-06    50.0
1970-01-07    52.0
1970-01-08    56.0
1970-01-09    54.0
1970-01-10    57.0
Name: tmax, dtype: float64

The sequence has 10 elements.  The first sequence element (at time step 0) is `60`.  The second sequence element (at time step 1) is `52`, and so on.


In [2]:
# Turn our sequence into a single row of data
data["tmax"].head(10).to_numpy()[np.newaxis,:]

array([[60., 52., 52., 53., 52., 50., 52., 56., 54., 57.]])

## Full Forward Pass

In this section, we will construct a complete forward pass for a Recurrent Neural Network (RNN). The process begins with the initialization of weights and biases, ensuring that bias terms are incorporated into both the hidden and output layers.

To ensure compatibility with the $\tanh$ activation function, the weights and biases will be appropriately scaled. Specifically:

- **Input and hidden weights**: These are initialized with small values to prevent the $\tanh$ nonlinearity from saturating, where it squashes inputs to either `1` or `-1`.
- **Output weights**: These are initialized with larger values, compensating for the fact that the hidden layer outputs are constrained to the range `[-1, 1]`.

While a complete RNN can learn optimal parameters through training, initializing weights and biases within these ranges facilitates the gradient descent process, improving convergence behavior.


In [15]:
np.random.seed(0)

# Define our weights and biases
# Scale them down so values get through the tanh nonlinearity
i_weight = np.random.rand(1,5) / 5 - .1
h_weight = np.random.rand(5,5) / 5 - .1
h_bias = np.random.rand(1,5) / 5 - .1

# Tanh pushes values to between -1 and 1, so scale up the output weights
o_weight = np.random.rand(5,1) * 50
o_bias = np.random.rand(1,1)

Then we can write the forward pass as a for loop.  This loop will process sequence elements one by one.  We'll store the output prediction and the hidden state:

In [150]:
# An array to store the output predictions
outputs = np.zeros(3)
# An array to store hidden states for use in backpropagation
hiddens = np.zeros((3, 5))

# This will store the previous hidden state, since we'll need it to calculate the current hidden step
prev_hidden = None
sequence = data["tmax"].tail(3).to_numpy()

for i in range(3):
    # Get the input sequence at the given position
    x = sequence[i].reshape(1,1)

    # Multiply input by input weight
    xi = x @ i_weight
    if prev_hidden is not None:
        # Add previous hidden to input
        xh = xi + prev_hidden @ h_weight + h_bias
    else:
        xh = xi

    # Apply our activation function
    xh = np.tanh(xh)
    prev_hidden = xh
    hiddens[i,] = xh

    # Multiply by the output weight
    xo = xh @ o_weight + o_bias
    outputs[i] = xo

A lot of the above step is very similar to backpropagation in a dense neural network.  The main difference comes in the next sequence position (1) where we need to consider multiple gradients at the hidden step:

In [159]:
l1_grad = loss_grad[1].reshape(1,1)

o_weight_grad += hiddens[1][:,np.newaxis] @ l1_grad
o_bias_grad += np.mean(l1_grad)

h1_grad = l1_grad @ o_weight.T

# We do have a next sequence position (2), so we need to include that gradient
# We multiply the h2 gradient by the weight to pull it back to the current sequence position
h1_grad += h2_grad @ h_weight.T

# The rest of the operation is the same
tanh_deriv = 1 - hiddens[1,:][np.newaxis,:] ** 2
h1_grad = np.multiply(h1_grad, tanh_deriv)

h_weight_grad += hiddens[1,:][:,np.newaxis] @ h1_grad
h_bias_grad += np.mean(h1_grad)

i_weight_grad += sequence[1].reshape(1,1).T @ h1_grad

Now, we can do the final sequence position, 0.  The main difference here is that we don't update the hidden gradient, since there is no previous sequence position that gave us hidden state input in the forward pass:

In [160]:
l0_grad = loss_grad[0].reshape(1,1)

o_weight_grad += hiddens[0][:,np.newaxis] @ l0_grad
o_bias_grad += np.mean(l0_grad)

h0_grad = l0_grad @ o_weight.T

h0_grad += h1_grad @ h_weight.T

tanh_deriv = 1 - hiddens[0,:][np.newaxis,:] ** 2
h0_grad = np.multiply(h0_grad, tanh_deriv)

# We don't update the hidden weight, since there was no previous hidden state
# We can update the hidden bias if you want

i_weight_grad += sequence[0].reshape(1,1).T @ h0_grad

We can now look at our gradient updates:

In [161]:
i_weight_grad

array([[-154.9774425 ,  -61.87971495,  -11.63213862,  -81.30882966,
          17.37148576]])

## Backward Pass

With the forward pass implemented, the next step is to consider the backward pass for updating the model parameters. The primary complexity in the backward pass arises from the fact that each parameter influences both the current output and future outputs.

### Key Insights

1. **Final Time Step**:  
   At the last sequence item, the hidden state only impacts the output of the final time step. This simplifies the gradient computation for this specific step.

2. **Intermediate Time Steps**:  
   In contrast, the hidden state at an earlier time step (e.g., time step 2) contributes to:
   - The output of the current time step.
   - The hidden state of the subsequent time step, which, in turn, impacts all future outputs.

### Implications for Backpropagation

During backpropagation, this dependency means that some parameters influence multiple outputs across the sequence. Consequently, the gradients for these parameters must accumulate contributions from all relevant outputs. Properly aggregating these gradients ensures accurate updates to the model parameters.


In [155]:
next_hidden = None

o_weight_grad, o_bias_grad, h_weight_grad, h_bias_grad, i_weight_grad = [0] * 5

for i in range(2, -1, -1):
    l_grad = loss_grad[i].reshape(1,1)

    o_weight_grad += hiddens[i][:,np.newaxis] @ l_grad
    o_bias_grad += np.mean(l_grad)

    o_grad = l_grad @ o_weight.T

    # Only add in the hidden gradient if a next sequence exists
    if next_hidden is not None:
        h_grad = o_grad + next_hidden @ h_weight.T
    else:
        h_grad = o_grad

    tanh_deriv = 1 - hiddens[i,:][np.newaxis,:] ** 2
    h_grad = np.multiply(h_grad, tanh_deriv)

    next_hidden = h_grad

    # Don't update the hidden weights for the first sequence position
    if i > 0:
        h_weight_grad += hiddens[i-1,:][:,np.newaxis] @ h_grad
        h_bias_grad += np.mean(h_grad)

    i_weight_grad += sequence[i].reshape(1,1).T @ h_grad

We can then use gradient descent to make parameter updates:

In [25]:
lr = 1e-6
# We'll divide the learning rate by the sequence length, since we were adding together the gradients
# This makes training the model more stable
lr = lr / 3

i_weight -= i_weight_grad * lr
h_weight -= h_weight_grad * lr
h_bias -= h_bias_grad * lr
o_weight -= o_weight_grad * lr
o_bias -= o_bias_grad * lr

## Full Implementation of an RNN

A full implementation of a Recurrent Neural Network (RNN) involves several critical steps, combining both the forward pass and backward pass to enable the network to process sequential data and learn meaningful patterns. Below, we outline the implementation process.

---

### 1. **Initialization**
Before running the network, we need to initialize its parameters:

- **Weights and Biases**:
  - **Input weights**: Small values to prevent saturation of the $\tanh$ activation function.
  - **Hidden weights**: Small values for stable hidden state updates.
  - **Output weights**: Larger values to account for the smaller range of hidden states (`[-1, 1]`).
  - **Bias terms**: Initialized to small values to ensure the model starts with a balanced activation.

This initialization helps stabilize training by ensuring gradients are well-behaved at the beginning of training.

---

### 2. **Forward Pass**

The forward pass computes the outputs for the entire input sequence, step by step. This involves:

1. **Input Processing**:
   At each time step, the input vector is multiplied by the input weights.

2. **Hidden State Update**:
   The hidden state is computed using the formula:
   \[
   h_t = \tanh(W_{ih} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h)
   \]
   where:
   - \( h_t \) is the current hidden state.
   - \( x_t \) is the input at time \( t \).
   - \( W_{ih} \) and \( W_{hh} \) are the input-to-hidden and hidden-to-hidden weights, respectively.
   - \( b_h \) is the bias for the hidden layer.

3. **Output Computation**:
   The output at each time step is computed as:
   \[
   y_t = W_{ho} \cdot h_t + b_o
   \]
   where \( W_{ho} \) is the hidden-to-output weight matrix, and \( b_o \) is the output bias.

---

### 3. **Backward Pass**

The backward pass adjusts the parameters based on the loss function by propagating the error gradients back through time. This process includes:

1. **Loss Calculation**:
   Compute the loss function (e.g., mean squared error or cross-entropy loss) based on the predicted outputs \( y_t \) and the true labels.

2. **Gradient Computation**:
   Gradients are computed using the chain rule, considering that:
   - Parameters in the **final hidden state** affect only the last output.
   - Parameters in **earlier hidden states** affect both the current output and all future outputs.

3. **Gradient Accumulation**:
   For parameters influencing multiple outputs (e.g., hidden-to-hidden weights), gradients from all affected outputs are accumulated.

4. **Weight Updates**:
   Using an optimization algorithm (e.g., stochastic gradient descent), weights and biases are updated:
   \[
   \theta \gets \theta - \eta \cdot \nabla_\theta
   \]
   where:
   - \( \theta \) represents the parameters.
   - \( \eta \) is the learning rate.
   - \( \nabla_\theta \) is the gradient of the loss with respect to \( \theta \).

---

### 4. **Training Loop**

The RNN is trained iteratively over multiple epochs. At each epoch:
- The forward pass computes predictions for the entire sequence.
- The backward pass computes gradients and updates the parameters.
- The loss is monitored to ensure convergence.

---

### 5. **Challenges and Solutions**

1. **Vanishing/Exploding Gradients**:
   - Gradients may vanish or explode during backpropagation through time (BPTT). Solutions include:
     - Gradient clipping to cap large gradients.
     - Using gated architectures like LSTMs or GRUs.

2. **Initialization Sensitivity**:
   - Proper initialization (as discussed) prevents instability during training.

3. **Sequence Length**:
   - For very long sequences, truncated BPTT can be used to limit gradient propagation to a manageable window.

---

### 6. **Inference**

After training, the RNN can be used for inference. Given an input sequence, the forward pass computes the outputs, which can be used for tasks like:
- Sequence-to-sequence translation.
- Time-series forecasting.
- Text generation.

---

By combining these steps, an RNN can be fully implemented to process sequential data, learn temporal dependencies, and make predictions.


In [26]:
from sklearn.preprocessing import StandardScaler
import math

# Define predictors and target
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

# Scale our data to have mean 0
scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

# Split into train, valid, test sets
np.random.seed(0)
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

Then we can initialize our weights and biases.  We'll scale our parameters so they are relatively small.  This helps the network descend better:

In [27]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

Then we'll write a forward pass:

In [28]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

And a backward pass:

In [29]:
def backward(layers, x, lr, grad, hiddens):
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            h_grad = out_grad @ o_weight.T

            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # So we take the output of tanh (next hidden state), and plug in
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            if j > 0:
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad

        # Normalize lr by number of sequence elements
        lr = lr / x.shape[0]
        i_weight -= i_weight_grad * lr
        h_weight -= h_weight_grad * lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]
    return layers

We can end by setting up a training loop and measuring error:

In [162]:
epochs = 250
lr = 1e-5

layer_conf = [
    {"type":"input", "units": 3},
    {"type": "rnn", "hidden": 4, "output": 1}
]
layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0
    for j in range(train_x.shape[0] - sequence_len):
        seq_x = train_x[j:(j+sequence_len),]
        seq_y = train_y[j:(j+sequence_len),]
        hiddens, outputs = forward(seq_x, layers)
        grad = mse_grad(seq_y, outputs)
        params = backward(layers, seq_x, lr, grad, hiddens)
        epoch_loss += mse(seq_y, outputs)

    if epoch % 50 == 0:
        sequence_len = 7
        valid_loss = 0
        for j in range(valid_x.shape[0] - sequence_len):
            seq_x = valid_x[j:(j+sequence_len),]
            seq_y = valid_y[j:(j+sequence_len),]
            _, outputs = forward(seq_x, layers)
            valid_loss += mse(seq_y, outputs)

        print(f"Epoch: {epoch} train loss {epoch_loss / len(train_x)} valid loss {valid_loss / len(valid_x)}")

Epoch: 0 train loss 3122.594400144508 valid loss 2171.3186862102025
Epoch: 50 train loss 30.593193275313595 valid loss 30.568271740103427
Epoch: 100 train loss 25.263986813543738 valid loss 24.435517510355645
Epoch: 150 train loss 22.9567624295313 valid loss 22.177010971976852
Epoch: 200 train loss 22.306774327704215 valid loss 21.557992202834164
